In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/to_zip/"
path_corpus_data = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"
bbn_path = base_dir + "BBN/BBN/"

Mounted at /content/drive/


In [8]:
import json

with open(bbn_path + "train.json", 'r') as inp:
    examples_train = [json.loads(l) for l in inp.readlines()]

In [9]:
print("il dataset train contiene ",len(examples_train), "elementi")

il dataset train contiene  32739 elementi


In [7]:
# print(len(examples_train[14535]['mentions']))
# print(len(examples_train[14535]['tokens']))
# prod = 1
# sum=0
# for elem in examples_train[14535]['mentions']:
#    prod *= len(elem)
#    sum+=len(elem)

# print(prod)
# print(sum/len(examples_train[14535]['mentions']))

27
77
7625597484987
3.0


In [ ]:
#del examples_train[14000:15000]

In [10]:
# rimuovo elemento 14535, caso problematico in cui generare tutte le permutazioni della frase avrebbe generato 7625597484987 frasi.
# questo elemento è una frase con 77 tokens, di cui 27 menzioni con ciascuna in media 3 tipi
del examples_train[14535]
examples_iter = iter(examples_train)

In [ ]:
#examples_iter = iter(examples_train[len(examples_train)//2:len(examples_train)+1])

In [11]:
import operator
import itertools
from tqdm import tqdm
from gensim.parsing.preprocessing import remove_stopwords

fp = open(path_corpus_data + 'corpus_data_bbn_entire_def.txt', "w")
restricted_set = set()

for i in tqdm(examples_iter):

  istance = i['mentions'][:]
  if len(istance)>1:

    d = sorted(istance, key=operator.itemgetter("start"))

    d = {k: list(v) for k, v in itertools.groupby(d, key=operator.itemgetter("start"))}
    all_list=[]
    for key, value in d.items():
      all_list.append(value)

    permutations = list(itertools.product(*all_list))

    iter = 0
    for p in permutations:
          mini_list = []
          for lista_labels in p:
            mini_list.append(lista_labels['labels'])

          permutations_2 = itertools.product(*mini_list)

          for p_2 in permutations_2:
            
            phrase = i['tokens'][:]
            
            num_pop=0
            for h in range(0, len(p_2)):
              start = permutations[iter][h]['start']
              end = permutations[iter][h]['end']
      
              phrase[start-num_pop:end-num_pop] = [p_2[h]]
              if((end-start)>1):
                num_pop += (end - start) - 1

              restricted_set.add(p_2[h])

              
            fp.write((remove_stopwords(' '.join(phrase)) + '\n'))
          iter+=1
  else:
    for k in istance[0]['labels']:
      phrase = i['tokens'][:]
      if (i['mentions'][0]['end'] - i['mentions'][0]['start'] ==1):          
        phrase[i['mentions'][0]['start']] = k
        restricted_set.add(k)
      else:
        phrase[i['mentions'][0]['start']:i['mentions'][0]['end']] = [k]
        restricted_set.add(k)

      fp.write((remove_stopwords(' '.join(phrase)) + '\n'))

fp.close()

32738it [00:26, 1236.93it/s]


In [12]:
restricted_list = list(restricted_set)

In [15]:
import pickle
with open(path_corpus_data + 'restricted_list_bbn.pkl', 'wb') as fp:
  pickle.dump(restricted_list, fp)